# The purpose of this notebook is to perform some preliminary data analysis and manipulation
## It is followed by more in-depth visual analysis in the eda_visuals.ipynb notebook and by more in-depth data transformations in utils/Dataloader.py

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os; os.getcwd()

# 

In [8]:
df = pd.read_csv('../data/3month_sample_more_categories.csv')

print(f'Rows: {len(df)}')
print(f'Columns: {len(df.columns)}')

Rows: 338587
Columns: 18


In [9]:
df.head(3)

,classified_id,listed_at_date,product_id,user_id,classified_price,retail_price,listing_price,favourites,viewed_count,state,brand_name,condition_name,size_name,color_name,category_name,subcategory_name,subsubcategory_name,subsubsubcategory_name
0,30343099,2023-09-06,954766,2425635,900,3000.0,1299,10,145,10,Air Jordan,Almost as new,Size 41,Black,Men,Men,Men,Sneakers
1,30346312,2023-09-06,484749,144602,225,750.0,350,12,119,10,Ganni,Almost as new,Size 38,Multi,Women,Women,Clothes,Skirts
2,30364278,2023-09-07,4402090,2028837,120,NaN,120,38,209,10,One Vintage,Good but used,Size M,Multi,Women,Women,Women,Women


In [10]:
# drop cols product_id and col state
df = df.drop(['product_id', 'state'], axis=1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338587 entries, 0 to 338586
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   classified_id           338587 non-null  int64  
 1   listed_at_date          338587 non-null  object 
 2   user_id                 338587 non-null  int64  
 3   classified_price        338587 non-null  int64  
 4   retail_price            132639 non-null  float64
 5   listing_price           338587 non-null  int64  
 6   favourites              338587 non-null  int64  
 7   viewed_count            338587 non-null  int64  
 8   brand_name              305786 non-null  object 
 9   condition_name          338587 non-null  object 
 10  size_name               265161 non-null  object 
 11  color_name              305690 non-null  object 
 12  category_name           338587 non-null  object 
 13  subcategory_name        338587 non-null  object 
 14  subsubcategory_name 

In [12]:
df['listed_at_date'] = pd.to_datetime(df['listed_at_date']).dt.date

In [14]:
df.isna().sum()

classified_id                  0
listed_at_date                 0
user_id                        0
classified_price               0
retail_price              204109
listing_price                  0
favourites                     0
viewed_count                   0
brand_name                 31875
condition_name                 0
size_name                  72332
color_name                 32342
category_name                  0
subcategory_name               0
subsubcategory_name            0
subsubsubcategory_name         0
dtype: int64

In [15]:
# get columns with nans
columns_with_nan = []
columns_with_few_nan = []

for i in df.columns:
    if df[i].isna().sum() > 50000:
        columns_with_nan.append(i)
    elif df[i].isna().sum() > 0:
        columns_with_few_nan.append(i)

print(columns_with_nan)
print(columns_with_few_nan)

['retail_price', 'size_name']
['brand_name', 'color_name']


In [16]:
# remove columns with too many nans
df = df.drop(columns_with_nan,axis=1)

print(f'Rows: {len(df)}')
print(f'Columns: {len(df.columns)}')

Rows: 336120
Columns: 14


In [17]:
"retail_price" in df.columns
"brand_name" in df.columns

True

In [18]:
brand_count = df.groupby(['brand_name']).count()['classified_price']

# check for brands with few listings
brands_to_drop = []
brands_to_drop_count = []
brands_to_keep = []
brands_to_keep_count = []
for x in brand_count.index:
    if brand_count[x] < 100:
        brands_to_drop.append(x)
        brands_to_drop_count.append(brand_count[x])
    else:
        brands_to_keep.append(x)
        brands_to_keep_count.append(brand_count[x])

keep = pd.DataFrame({'brand':brands_to_keep,'count_instances':brands_to_keep_count})
drop = pd.DataFrame({'brand':brands_to_drop,'count_instances':brands_to_drop_count})

drop

,brand,count_instances
0,"""E""",1
1,&Copenhagen,3
2,&Denim,11
3,&klevering,1
4,&tradition,24
...,...,...
4757,Öhouse,40
4758,Önling,2
4759,ØLÅF,4
4760,ØST London,2


In [19]:
# exclude rare brands
df = df[~df['brand_name'].isin(brands_to_drop)].copy()

print(f'Rows: {len(df)}')
print(f'Columns: {len(df.columns)}')

Rows: 286780
Columns: 14


In [21]:
df.head(3)

,classified_id,listed_at_date,user_id,classified_price,listing_price,favourites,viewed_count,brand_name,condition_name,color_name,category_name,subcategory_name,subsubcategory_name,subsubsubcategory_name
0,30343099,2023-09-06,2425635,900,1299,10,145,Air Jordan,Almost as new,Black,Men,Men,Men,Sneakers
1,30346312,2023-09-06,144602,225,350,12,119,Ganni,Almost as new,Multi,Women,Women,Clothes,Skirts
2,30364278,2023-09-07,2028837,120,120,38,209,One Vintage,Good but used,Multi,Women,Women,Women,Women


# Adjust scale on quantitative columns

In [22]:
scaler = StandardScaler()
df['classified_price_standardized'] = scaler.fit_transform(df[['classified_price']])
df['viewed_count_standardized'] = scaler.fit_transform(df[['viewed_count']])
df['favourites_standardized'] = scaler.fit_transform(df[['favourites']])

# Normalizing the columns
min_max_scaler = MinMaxScaler()
df['classified_price_normalized'] = min_max_scaler.fit_transform(df[['classified_price']])
df['viewed_count_normalized'] = min_max_scaler.fit_transform(df[['viewed_count']])
df['favourites_normalized'] = min_max_scaler.fit_transform(df[['favourites']])


In [25]:
# df_with_dummies.to_csv('../data/post_preprocessing_with_dummies.csv',index=False)
# df.to_csv('../data/post_preprocessing_without_dummies.csv',index=False)